# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv("../WeatherPy/output_data/cleancitydata.csv")
weather_data_df

,city,country,latitude,longitude,temp,humid,cloud,wind
0,Yangi Marg`ilon,UZ,40.43,71.72,78.80,57,40,2.24
1,São Filipe,CV,14.90,-24.50,74.35,83,68,13.22
2,Torbay,CA,47.67,-52.73,55.99,100,90,17.22
3,Albany,US,42.60,-73.97,73.00,97,94,4.38
4,Nanjing,CN,32.06,118.78,77.00,100,90,4.47
...,...,...,...,...,...,...,...,...
550,Bintulu,MY,3.17,113.03,87.80,74,20,6.93
551,Esim,GH,4.87,-2.24,74.01,86,19,9.13
552,Syedove,UA,47.08,38.16,73.99,100,91,8.01
553,Doha,QA,25.29,51.53,102.20,37,41,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
weather_data_df = weather_data_df.dropna()

locations = weather_data_df[["latitude", "longitude"]].astype(float)
humidity = weather_data_df["humid"].astype(float)

In [5]:
fig = gmaps.figure(center=(30, 20), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(
    locations, weights=humidity, 
    max_intensity=weather_data_df["humid"].max(), 
    dissipating=False, 
    point_radius = 1)

fig.add_layer(heat_layer)
    
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
my_ideal_weather_locations_df = weather_data_df.loc[(weather_data_df["temp"] > 65) & \
                                                    (weather_data_df["temp"] < 75) & \
                                                    (weather_data_df["humid"] < 80) & \
                                                    (weather_data_df["humid"] > 50) & \
                                                    (weather_data_df["cloud"] < 10) & \
                                                    (weather_data_df["wind"] < 10)]
my_ideal_weather_locations_df

,city,country,latitude,longitude,temp,humid,cloud,wind
51,Sioux Lookout,CA,50.10,-91.92,68.00,64,5,9.17
64,Imeni Babushkina,RU,59.76,43.13,71.06,57,1,2.15
97,Sheltozero,RU,61.37,35.37,65.64,76,5,6.38
140,Redmond,US,47.67,-122.12,73.99,73,0,3.78
147,Bismarck,US,46.81,-100.78,70.00,56,1,4.70
154,Itaituba,BR,-4.28,-55.98,72.73,64,1,1.23
190,Ninotsminda,GE,41.26,43.59,69.80,64,0,2.24
197,Novorzhev,RU,57.03,29.33,66.88,68,1,4.38
251,Farafangana,MG,-22.82,47.83,65.70,71,0,4.18
262,São Félix do Xingu,BR,-6.64,-51.99,67.42,65,0,1.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = my_ideal_weather_locations_df[["latitude", "longitude", "city", "country"]]
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.rename(columns= {"latitude": "Lat", "longitude": "Lng", "city": "City", "country": "Country"})

hotel_df

C:\Users\Mama\anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Lat,Lng,City,Country,Hotel Name
51,50.10,-91.92,Sioux Lookout,CA,
64,59.76,43.13,Imeni Babushkina,RU,
97,61.37,35.37,Sheltozero,RU,
140,47.67,-122.12,Redmond,US,
147,46.81,-100.78,Bismarck,US,
154,-4.28,-55.98,Itaituba,BR,
190,41.26,43.59,Ninotsminda,GE,
197,57.03,29.33,Novorzhev,RU,
251,-22.82,47.83,Farafangana,MG,
262,-6.64,-51.99,São Félix do Xingu,BR,


In [8]:
# assemble url and make API request
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

#    print(f"Retrieving Results for Index: {index}, hotel in {row['City Name']}")
    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "hotel"
    target_radius = 50000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
        }
    
    # Pull data from API
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        hotel_df.loc[index, "Lat"] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = results[0]["geometry"]["location"]["lng"]
           
    except (KeyError, IndexError):
        pass

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Display figure
fig = gmaps.figure(center=(30, 20), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(
    locations, 
    dissipating=False, 
    point_radius = 1)

fig.add_layer(heat_layer)

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))